In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax
from tqdm import tqdm, trange

In [1]:
# from datasets import load_dataset
# dataset = load_dataset('breadlicker45/youtube-comments-180k', split='train')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [2]:
# dataset.to_csv('youtube-comments-180k.csv')

Creating CSV from Arrow format:   0%|          | 0/188 [00:00<?, ?ba/s]

32084224

In [4]:
yt_df = pd.read_csv("youtube-comments-180k.csv")
yt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187482 entries, 0 to 187481
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    187480 non-null  object
dtypes: object(1)
memory usage: 1.4+ MB


In [12]:
# train, test = train_test_split(yt_df, test_size=0.35)
# train.info()
# test.info()
# train.to_csv('train_yt.csv')
# test.to_csv('test_yt.csv')

<class 'pandas.core.frame.DataFrame'>
Index: 121863 entries, 31875 to 110863
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    121861 non-null  object
dtypes: object(1)
memory usage: 1.9+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 65619 entries, 56143 to 179671
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    65619 non-null  object
dtypes: object(1)
memory usage: 1.0+ MB


# LeetSpeak processing

In [6]:
#load existing leet dictionary to initiate
import json
leet_fp = {}
with open('../accuracy/pyleetspeak_leetDict.json') as json_file:
    leet_fp = json.load(json_file)

leet_fp

{'A': {'0': 372, '3': 402, '4': 266, '@': 284, '_': 278, '1': 348},
 'B': {'0': 40, '4': 20, '3': 38, '1': 36, '_': 40, '@': 20},
 'C': {'3': 35, '@': 26, '4': 12, '_': 24, '1': 29, '0': 38},
 'D': {'0': 46, '3': 47, '4': 33, '@': 33, '_': 51, '1': 34},
 'E': {'3': 514, '0': 299, '1': 245, '_': 216, '@': 145, '4': 158},
 'F': {'0': 29, '@': 23, '_': 27, '1': 21, '3': 23, '4': 13},
 'G': {'3': 53, '_': 41, '0': 45, '1': 39, '@': 30, '4': 18},
 'H': {'3': 34, '0': 44, '@': 24, '_': 28, '4': 23, '1': 29},
 'I': {'1': 358, '0': 254, '@': 120, '3': 246, '4': 107, '_': 190},
 'J': {'3': 11, '_': 10, '1': 8, '@': 7, '4': 3, '0': 9},
 'K': {'0': 26, '3': 23, '@': 22, '_': 18, '4': 11, '1': 31},
 'L': {'0': 69, '3': 61, '@': 44, '_': 38, '1': 53, '4': 29},
 'M': {'4': 31, '3': 41, '@': 33, '_': 45, '1': 41, '0': 48},
 'N': {'0': 74, '3': 83, '@': 37, '4': 42, '_': 61, '1': 57},
 'O': {'0': 397, '3': 289, '@': 134, '1': 265, '4': 146, '_': 211},
 'P': {'4': 30, '3': 36, '_': 38, '1': 33, '@': 48

In [9]:
import LeetMining as lm
lm.setupLeetDict(leet_dict=leet_fp)

In [14]:
# SMALL TEST WITH LEET CODE: Run Roberta and leet decode algorithm on train data to get analysis post leet substitution
testLeet = {'keyword':[], 'reply':[], 'leetWords':[], 'candidates':[]}

for i in trange(100):
    testLeet['keyword'].append(train['keyword'].values[i])
    testLeet['reply'].append(train['reply'].values[i])

    leetWords = lm.getLeetWordList(train['reply'].values[i])
    testLeet['leetWords'].append(leetWords)
    candidates = []
    for lword in leetWords:
        possibleMatches = lm.getMatchList(lword)
        candidates.append(possibleMatches)
        possibleSubs = lm.getPossibleSubstitutions(lword, possibleMatches)
        lm.updateLeetDict(possibleSubs)
    testLeet['candidates'].append(candidates)




100%|██████████| 100/100 [00:08<00:00, 11.16it/s]


In [15]:
testLeet_df = pd.DataFrame(testLeet)
testLeet_df.head()

,keyword,reply,leetWords,candidates
0,Dogecoin,You'll be degrading your empowerment and dissi...,[],[]
1,Ukraine,Russia is the one with a uniquely bad NAZI pro...,[],[]
2,weather,"be thankful thats all you got , we got tornado...",[],[]
3,Queen Elizabeth,What a surprise - not,[],[]
4,Vaccine,"No, actually the doctors hope that “you’ll “ b...",[],[]


In [8]:
testLeet_df['reply'].values[3]

'What a surprise - not'

In [16]:
# testLeet_df.query('leetWords ')
# testLeet_df[testLeet_df['leetWords'].size >0]
# len(testLeet_df['leetWords'].values)

#df['CreationDate'].str.len() -> gives length of arrays stored in cell https://stackoverflow.com/questions/41340341/how-to-determine-the-length-of-lists-in-a-pandas-dataframe-column
testLeet_df[testLeet_df['leetWords'].str.len() >0] #just the rows with leetspeak

,keyword,reply,leetWords,candidates
7,Bitcoin,what the fuck b*tc**n is legal tender in el sa...,[b*tc**n],[[]]
9,World Cup,So what about the 1000s England has killed and...,[1000s],[[]]
14,COVID-19,Canada need to Pay attention to weak peoples a...,[160Vanderhoof],[[]]
21,COVID-19,I’m retweeting this again due to variants XBB....,"[XBB.1.5, (origin, York), BF.7, (origin, China...","[[], [origin], [york], [], [origin], [china], ..."
60,nba,The only rookie &gt;&gt; Keegan Murray is Paol...,[gt;gt;],[[]]
67,Dogecoin,"Wow, this might be the most valuable remaining...","[FTX(Though, that)]","[[], [that, thats]]"
70,Elon Musk,In case you missed it:😂,[it:],"[[its, itu, ito, itt, ity, itl, ita, iti, it, ..."
73,Dogecoin,While Twitter cooks...you could commercializet...,"[cooks...you, of:1, BusinessNews2]","[[], [ofa, offa, of, ofr, offs, offi, off, oft..."
76,Dogecoin,"This didn’t age well , right? How many followe...",[loose#SHIB],[[]]
78,TikTok,Calling early-stage founders! Awesome opportun...,"[early-stage, amp;]","[[], [amp, amps]]"


In [10]:
print(lm.getLeetDict())

{'A': {':': 1, '1': 1, '/': 1, '2': 3, '0': 10}, 'B': {'2': 3, '0': 1}, 'C': {}, 'D': {'2': 3}, 'E': {':': 1, '/': 1, '0': 6, '2': 1}, 'F': {':': 3, '2': 1}, 'G': {}, 'H': {':': 1, '2': 3}, 'I': {':': 1, '1': 1, '/': 1, '0': 10, '2': 1}, 'J': {}, 'K': {}, 'L': {':': 1, '2': 3}, 'M': {'2': 3}, 'N': {'2': 2, '0': 2}, 'O': {':': 1, ';': 1, '2': 1, '0': 6}, 'P': {'2': 3}, 'Q': {}, 'R': {'0': 2, '2': 3}, 'S': {';': 5, ')': 2, ':': 1, '1': 1, '0': 1, '2': 4}, 'T': {':': 1, '$': 2, '0': 2, '2': 3}, 'U': {':': 1, '/': 2, '2': 2, '0': 2}, 'V': {'2': 2}, 'W': {}, 'X': {}, 'Y': {':': 1}, 'Z': {'2': 1}}


In [17]:
# train1, train2 = train_test_split(train, test_size=0.30)
# train1.reset_index(drop=True)
# train2.reset_index(drop=True)
# print("Training Dataset1:")
# train1.info()
# print("\nTraining Dataset2:")
# train2.info()

In [18]:
# Run leet algorithm on train data to extract leet words and their substitutions
lm.setupLeetDict() # initiate FP list
trainLeet = {'keyword':[], 'reply':[], 'leetWords':[], 'candidates':[]}

for i in trange(len(train['reply'].values)):
    trainLeet['keyword'].append(train['keyword'].values[i])
    trainLeet['reply'].append(train['reply'].values[i])

    leetWords = lm.getLeetWordList(train['reply'].values[i])
    trainLeet['leetWords'].append(leetWords)
    candidates = []
    for lword in leetWords:
        possibleMatches = lm.getMatchList(lword)
        candidates.append(possibleMatches)
        possibleSubs = lm.getPossibleSubstitutions(lword, possibleMatches)
        lm.updateLeetDict(possibleSubs)
    trainLeet['candidates'].append(candidates)

train1_result_df = pd.DataFrame(trainLeet)
train1_result_df.head()

100%|██████████| 119178/119178 [3:51:32<00:00,  8.58it/s]  


,keyword,reply,leetWords,candidates
0,Dogecoin,You'll be degrading your empowerment and dissi...,[],[]
1,Ukraine,Russia is the one with a uniquely bad NAZI pro...,[],[]
2,weather,"be thankful thats all you got , we got tornado...",[],[]
3,Queen Elizabeth,What a surprise - not,[],[]
4,Vaccine,"No, actually the doctors hope that “you’ll “ b...",[],[]


In [19]:
#collect training results - leet dict and the result table
import json
with open("twitter_leetDict.json", "w") as outfile: 
    json.dump(lm.getLeetDict(), outfile)

In [35]:
# import numpy as np
# leetDict['B']

# allKeysCounts = []
# allKeys = []
# for key in leetDict["B"].keys():
#     allKeys.append(key)
#     allKeysCounts.append(leetDict["B"][key])
    
#     #print(key, leetDict["F"][key])
# f_set = np.array(leetDict["B"])
# print(f_set)

# f_set_df = pd.DataFrame()

{'’': 1037, '8': 132, '?': 2, '*': 49, '(': 259, '7': 72, '0': 163, '3': 282, '$': 443, '"': 201, '2': 317, '‘': 41, ')': 78, '“': 151, '1': 468, '5': 167, '-': 38, ':': 88, '…': 208, '.': 60, ',': 1, '⁵': 11, '9': 95, '4': 108, '6': 60, "'": 45, '{': 1, '£': 8, ';': 6, '!': 2, '»': 2, '+': 18, '\xa0': 3, '/': 14, '”': 5, '=': 3, '²': 10, '⁷': 6, '⁸': 6, '`': 1, '[': 12, '_': 1, '—': 10, '^': 7, ']': 1}


In [20]:
# train1_result_df.to_csv('train1_result.csv', index=False)
# train1.to_csv('train1.csv', index=False)
train1_result_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119178 entries, 0 to 119177
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   keyword     119178 non-null  object
 1   reply       119178 non-null  object
 2   leetWords   119178 non-null  object
 3   candidates  119178 non-null  object
dtypes: object(4)
memory usage: 3.6+ MB


In [21]:
#print leetword rows only
train1_result_df[train1_result_df['leetWords'].str.len() >0]

,keyword,reply,leetWords,candidates
7,Bitcoin,what the fuck b*tc**n is legal tender in el sa...,[b*tc**n],[[]]
9,World Cup,So what about the 1000s England has killed and...,[1000s],[[]]
14,COVID-19,Canada need to Pay attention to weak peoples a...,[160Vanderhoof],[[]]
21,COVID-19,I’m retweeting this again due to variants XBB....,"[XBB.1.5, (origin, York), BF.7, (origin, China...","[[], [origin], [york], [], [origin], [china], ..."
60,nba,The only rookie &gt;&gt; Keegan Murray is Paol...,[gt;gt;],[[]]
...,...,...,...,...
119160,Dogecoin,It will get a top-notch care,[top-notch],[[]]
119163,Vaccine,"#ErasingWomenWomen, pregnant women.People incl...",[women.People],[[]]
119167,nba,$mommie0198 I absolutely love and adore my 2 d...,[$mommie0198],[[]]
119168,weather,At least A&amp;W gets it. I quit buying Coke c...,"[Aamp;W, Aamp;W]","[[], []]"


In [22]:
print(lm.getLeetDict())

{'A': {':': 969, '1': 3934, '/': 401, '2': 2548, '0': 7179, '(': 571, '5': 2972, ')': 540, '$': 981, '8': 1091, '3': 1889, '6': 620, '*': 481, '9': 1480, '-': 447, '7': 728, '4': 1154, '.': 706, ';': 805, '£': 13, '[': 31, '«': 136, '\xa0': 207, '=': 27, '{': 2, '–': 1, '!': 130, '+': 62, '¿': 12, '²': 52, '⁷': 18, '⁸': 18, '_': 11, '☭': 7, '—': 51, '»': 3, '#': 19, '\u202f': 1, '„': 4, '￼': 1, '^': 14, '🖒': 3, '%': 8, '→': 1, '？': 1, '~': 17, ']': 19, '。': 1, '¡': 1, '°': 3, '⁵': 18, '⁶': 18, '\u3000': 1, '↓': 1, '•': 1, '\u200b': 12}, 'B': {'2': 695, '0': 576, '(': 540, '/': 16, '$': 828, '1': 917, ':': 195, '3': 552, '5': 346, '9': 404, '7': 222, '=': 5, '*': 68, '6': 146, ')': 150, '8': 317, ';': 201, '⁵': 29, '[': 14, '4': 287, '.': 250, '\xa0': 39, '-': 56, '£': 12, '+': 20, '￼': 1, '`': 1, '!': 25, '⁷': 6, '²': 20, '⁸': 6, '—': 16, '»': 3, '#': 5, '{': 1, '🖒': 3, '♡': 1, '~': 4, ']': 1, '¡': 2, '«': 1, '⁶': 6, '_': 1, '^': 7, '\u200b': 2}, 'C': {'(': 568, '/': 13, '2': 441, '$':